<a href="https://colab.research.google.com/github/ntk-nguyen/NLP_notebook/blob/master/imdb_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [0]:
# Loading IMDB data from TensorFlow
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete85PK2I/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete85PK2I/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete85PK2I/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
# Splitting to training and test data
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences, training_labels = [], [] 
testing_sentences, testing_labels = [], []

for s, l in train_data:
  training_sentences.append(str(s.numpy))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(str(s.numpy))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
# Tokenizing
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sequences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? bound method eagertensorbase numpy of tf tensor shape dtype string numpy b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty p

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')                               
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5278 - accuracy: 0.7077 - val_loss: 0.3961 - val_accuracy: 0.8380
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2696 - accuracy: 0.8912 - val_loss: 0.3758 - val_accuracy: 0.8366
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1231 - accuracy: 0.9637 - val_loss: 0.4150 - val_accuracy: 0.8206
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0328 - accuracy: 0.9953 - val_loss: 0.4618 - val_accuracy: 0.8179
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0090 - accuracy: 0.9992 - val_loss: 0.5118 - val_accuracy: 0.8161
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0029 - accuracy: 0.9998 - val_loss: 0.5435 - val_accuracy: 0.8156
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.5791 - val_accuracy: 0.8172
Epoch 

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1440], [975], [5], [1546], [1546], [4720], [], [799], [2028], [11], [2937], [2193], [], [799], [2028], [11], [588], [], [11], [588], [], [5], [1791], [5], [4522], [11], [2937], [1284], [], [], [2028], [1012], [2937], [975], [588], [799], []]
